# **User 관심도 점수를 카드 혜택에 맞춰 카드별 매칭 점수 도출**
# 1. 필요한 라이브러리 및 데이터 불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
card = pd.read_csv('/content/drive/MyDrive/CARD-LANGCHAIN/data/card_data.csv')
interests = pd.read_csv('/content/drive/MyDrive/CARD-LANGCHAIN/data/관심도모델링1차데이터.csv')

# 2. 카드 혜택 대분류 카테고리 뽑기

In [28]:
card.head()

,Card Name,Annual_fee,Base Record,Category_List
0,하나 트래블로그 SKYPASS 신용카드,"국내 4만8천원, 해외 4만8천원, 가족겸용 없음",조건없음,['항공마일리지']
1,DA카드의정석Ⅱ,"국내 1만2천원, 해외 1만2천원",직전 1개월 합계 30만원 이상,"['쇼핑', '레저', '카페/베이커리', '편의점', '대형마트', '의료', '..."
2,삼성카드 & MILEAGE PLATINUM(스카이패스),"국내 4만7천원, 해외 4만9천원",조건없음,"['쇼핑', '주유', '카페/베이커리', '편의점', '대중교통', '간편결제',..."
3,현대카드ZERO Edition3(할인형),"국내 1만5천원, 해외 1만5천원, 가족 5천원",조건없음,"['대중교통', '간편결제', '편의점', '통신', '카페/베이커리', '대형마트..."
4,삼성카드 taptap O,"국내 1만원, 해외 1만원",직전 1개월 합계 30만원 이상,"['포인트/캐시백', '카페/베이커리', '쇼핑', '통신', '대중교통', '영화..."


In [29]:
import ast
card['Category_List'] = card['Category_List'].apply(ast.literal_eval)

In [30]:
category_dict = {
'항공' : '항공마일리지',
'쇼핑/간편결제' : ['쇼핑', '간편결제' , '포인트/캐시백', '편의점', '대형마트'],
'카페/베이커리' :['카페', '베이커리', '외식'],
'의료' : '의료'
,'반려동물': '반려동물'
,'뷰티': '뷰티'
,'교통/주유' : ['대중교통', '주유', '하이패스'],
'교육/육아': ['교육', '육아'],
'문화': ['문화', '레저', '영화'],
'납부': ['통신', '관리비', '납부 혜택']}

# 매핑 함수
def map_category_list_to_key(category_list):
  keys = set()
  for category in category_list:
    for key, values in category_dict.items():
        if category in values:
            keys.add(key)
  return list(keys)

# 대분류 카테고리 컬럼 추가
card['category'] = card['Category_List'].apply(map_category_list_to_key)

card.head()

,Card Name,Annual_fee,Base Record,Category_List,category
0,하나 트래블로그 SKYPASS 신용카드,"국내 4만8천원, 해외 4만8천원, 가족겸용 없음",조건없음,[항공마일리지],[항공]
1,DA카드의정석Ⅱ,"국내 1만2천원, 해외 1만2천원",직전 1개월 합계 30만원 이상,"[쇼핑, 레저, 카페/베이커리, 편의점, 대형마트, 의료, 뷰티, 대중교통, 외식]","[의료, 문화, 교통/주유, 쇼핑/간편결제, 뷰티, 카페/베이커리]"
2,삼성카드 & MILEAGE PLATINUM(스카이패스),"국내 4만7천원, 해외 4만9천원",조건없음,"[쇼핑, 주유, 카페/베이커리, 편의점, 대중교통, 간편결제, 대형마트, 외식, 통...","[의료, 항공, 카페/베이커리, 반려동물, 문화, 납부, 교통/주유, 쇼핑/간편결제..."
3,현대카드ZERO Edition3(할인형),"국내 1만5천원, 해외 1만5천원, 가족 5천원",조건없음,"[대중교통, 간편결제, 편의점, 통신, 카페/베이커리, 대형마트, 주유, 외식, 쇼...","[의료, 반려동물, 교육/육아, 문화, 납부, 교통/주유, 쇼핑/간편결제, 뷰티, ..."
4,삼성카드 taptap O,"국내 1만원, 해외 1만원",직전 1개월 합계 30만원 이상,"[포인트/캐시백, 카페/베이커리, 쇼핑, 통신, 대중교통, 영화, 편의점]","[교통/주유, 쇼핑/간편결제, 문화, 납부]"


# 3. 혜택 별 카드 매칭

In [32]:
card['category']

,category
0,[항공]
1,"[의료, 문화, 교통/주유, 쇼핑/간편결제, 뷰티, 카페/베이커리]"
2,"[의료, 항공, 카페/베이커리, 반려동물, 문화, 납부, 교통/주유, 쇼핑/간편결제..."
3,"[의료, 반려동물, 교육/육아, 문화, 납부, 교통/주유, 쇼핑/간편결제, 뷰티, ..."
4,"[교통/주유, 쇼핑/간편결제, 문화, 납부]"
...,...
161,"[의료, 교육/육아, 문화, 납부, 교통/주유, 쇼핑/간편결제, 뷰티, 카페/베이커리]"
162,[항공]
163,"[쇼핑/간편결제, 문화, 카페/베이커리]"
164,"[교통/주유, 쇼핑/간편결제]"


In [33]:
interests.head()

,user_id,category,암묵적관심도,명시적관심도
0,1,교육/육아,1,0
1,1,문화,0,1
2,1,납부,0,1
3,1,항공,0,1
4,1,반려동물,1,0


user_id별로 관심 카테고리가 있는 카드 혜택을 매칭시켜
암묵적 관심도는 *0.7, 명시적관심도 * 0.9 를 곱해
매칭 점수가 가장 큰 카드뽑기

In [ ]:
def map_category_list_to_key(category_list):
  keys = set()
  for category in category_list:
    for key, values in category_dict.items():
        if category in values:
            keys.add(key)
  return list(keys)

In [59]:
card

,Card Name,Annual_fee,Base Record,Category_List,category
0,하나 트래블로그 SKYPASS 신용카드,"국내 4만8천원, 해외 4만8천원, 가족겸용 없음",조건없음,[항공마일리지],[항공]
1,DA카드의정석Ⅱ,"국내 1만2천원, 해외 1만2천원",직전 1개월 합계 30만원 이상,"[쇼핑, 레저, 카페/베이커리, 편의점, 대형마트, 의료, 뷰티, 대중교통, 외식]","[의료, 문화, 교통/주유, 쇼핑/간편결제, 뷰티, 카페/베이커리]"
2,삼성카드 & MILEAGE PLATINUM(스카이패스),"국내 4만7천원, 해외 4만9천원",조건없음,"[쇼핑, 주유, 카페/베이커리, 편의점, 대중교통, 간편결제, 대형마트, 외식, 통...","[의료, 항공, 카페/베이커리, 반려동물, 문화, 납부, 교통/주유, 쇼핑/간편결제..."
3,현대카드ZERO Edition3(할인형),"국내 1만5천원, 해외 1만5천원, 가족 5천원",조건없음,"[대중교통, 간편결제, 편의점, 통신, 카페/베이커리, 대형마트, 주유, 외식, 쇼...","[의료, 반려동물, 교육/육아, 문화, 납부, 교통/주유, 쇼핑/간편결제, 뷰티, ..."
4,삼성카드 taptap O,"국내 1만원, 해외 1만원",직전 1개월 합계 30만원 이상,"[포인트/캐시백, 카페/베이커리, 쇼핑, 통신, 대중교통, 영화, 편의점]","[교통/주유, 쇼핑/간편결제, 문화, 납부]"
...,...,...,...,...,...
161,LOCA PLATINUM(할인형),"국내 10만원, 해외 10만원, 가족겸용 없음",조건없음,"[카페/베이커리, 쇼핑, 뷰티, 대형마트, 편의점, 주유, 외식, 영화, 통신, 교...","[의료, 교육/육아, 문화, 납부, 교통/주유, 쇼핑/간편결제, 뷰티, 카페/베이커리]"
162,I-MILEAGE(아시아나),"국내 4만1천원, 해외 4만2천원",직전 1개월 합계 50만원 이상,"[항공마일리지, 카페/베이커리]",[항공]
163,The CJ-현대카드M Edition2,"국내 1만5천원, 해외 2만원",조건없음,"[쇼핑, 영화, 외식, 카페/베이커리, 포인트/캐시백]","[쇼핑/간편결제, 문화, 카페/베이커리]"
164,KB국민 그린카드(서울형),"국내전용 없음, 해외겸용 없음",직전 1개월 합계 20만원 이상,"[포인트/캐시백, 대중교통, 쇼핑, 대형마트]","[교통/주유, 쇼핑/간편결제]"


In [85]:
card[
    'category'
][3][1]

'반려동물'

In [89]:
interests.head()

,user_id,category,암묵적관심도,명시적관심도
0,1,교육/육아,1,0
1,1,문화,0,1
2,1,납부,0,1
3,1,항공,0,1
4,1,반려동물,1,0


In [92]:
matching_card = []

def calculate
for idx, user_row in interests.iterrows():
  for i, card_row in card.iterrows():
    score = 0
    # 카테고리 매칭
    if user_row['category'] in card_row['category']:
      if user_row['암묵적관심도'] == 1:
         score += 0.7
      if user_row['명시적관심도'] == 1:
        score += 0.9

    # 결과 저장
    if score > 0:
      matching_card.append({
          'user_id': user_row['user_id'],
          'Card Name': card_row['Card Name'],
          'matching_score': score
      })
  return pd.DataFrame(matching_card)

1 하나 트래블로그 SKYPASS 신용카드 0
1 DA카드의정석Ⅱ 0
1 삼성카드 & MILEAGE PLATINUM(스카이패스) 0.7
1 현대카드ZERO Edition3(할인형) 1.4
1 삼성카드 taptap O 1.4
1 신한카드 Mr.Life 1.4
1 현대카드M 2.0999999999999996
1 신한카드 MY CAR 2.0999999999999996
1 현대백화점카드 2.0999999999999996
1 신한카드 Deep Oil 2.0999999999999996
1 현대 아멕스 골드 카드 Edition2 2.0999999999999996
1 LOCA 365 2.8
1 LOCA CLASSIC (로카 클래식) 3.5
1 KB국민 굿데이카드 4.2
1 우리 K-패스 4.9
1 국민행복 삼성카드 V2 5.6000000000000005
1 신한카드 처음(ANNIVERSE) 5.6000000000000005
1 신한카드 Point Plan 5.6000000000000005
1 롯데 국민행복카드 6.300000000000001
1 메리어트 본보이™ 더 베스트 신한카드 6.300000000000001
1 신한카드 처음 6.300000000000001
1 디지로카 Las Vegas 7.000000000000001
1 LOCA LIKIT 1.2 7.700000000000001
1 네이버페이 taptap 7.700000000000001
1 KB국민 굿데이올림카드 8.4
1 아시아나 신한카드 Air 1.5 8.4
1 삼성 iD ON 카드 8.4
1 신한카드 Simple+ 9.1
1 KB국민 다담카드 9.799999999999999
1 기업은행 BLISS.5 카드 9.799999999999999
1 현대카드MM 10.499999999999998
1 신한카드 경차사랑 Life 10.499999999999998
1 현대카드 Summit 11.199999999999998
1 신한카드 국민행복 11.899999999999997
1 신한카드 Air One 11.8999999999

KeyboardInterrupt: 

In [71]:
matching_card

,user_id,Card_Name,matching_score
2,1,삼성카드 & MILEAGE PLATINUM(스카이패스),0.7
3,1,현대카드ZERO Edition3(할인형),1.4
4,1,삼성카드 taptap O,1.4
5,1,신한카드 Mr.Life,1.4
6,1,현대카드M,2.1
...,...,...,...
8129,10,LOCA PLATINUM(할인형),69.3
8130,10,I-MILEAGE(아시아나),69.3
8131,10,The CJ-현대카드M Edition2,70.2
8132,10,KB국민 그린카드(서울형),70.2


In [93]:
card[card['Card Name']=='신한카드 Mr.Life']['category']

,category
5,"[의료, 납부, 교통/주유, 쇼핑/간편결제, 카페/베이커리]"


In [79]:
interests[interests['user_id']==1]

,user_id,category,암묵적관심도,명시적관심도
0,1,교육/육아,1,0
1,1,문화,0,1
2,1,납부,0,1
3,1,항공,0,1
4,1,반려동물,1,0
5,1,쇼핑/간편결제,1,0


In [77]:
pd.set_option('display.max_colwidth', None)